In [ ]:
# NegativeResultDetector

## SciBERT text classification model for positive and negative results prediction in scientific abstracts
## of clinical psychology and psychotherapy.

# 1. Load Data
## You can load your own data or utilize our example datasets. 
## Your data should be a single column 'csv' file with a column containing scientific abstracts.
## Make sure your text column is named 'text', otherwise replace 'text' within the preprocess function 
## with the name of your text column.  

## We present two options for loading your data:
from datasets import load_dataset

## Option 1: Using Github CSV File
## Insert Github 'raw' url for inference dataset 
example_url =    'https://github.com/PsyCapsLock/PubBiasDetect/blob/main/Data/Example_Data/example_df.csv?raw=true'
dataset = load_dataset('csv', data_files={'inference': example_url})

## Option 2: Using Local CSV File
dataset = load_dataset('csv', data_files={'inference': "example_folder/example_df.csv"})


# 2. Preprocessing
## Load tokenizer in uncased settings with scivocab
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

## preprocess function
## Make sure your text column is named 'text'. Otherwise replace 'text' with the name of your text column.
def preprocess_function(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=512,
                     padding='max_length'
                     )

## map preprocess_function to tokenized_data
tokenized_data = dataset.map(preprocess_function, batched=True)

# 3. Load Model
from transformers import Trainer, AutoModelForSequenceClassification
PubBiasDetect = AutoModelForSequenceClassification.from_pretrained("ClinicalMetaScience/NegativeResultDetector")

## Initialize the trainer with the model and tokenizer
trainer = Trainer(
    model=PubBiasDetect,
    tokenizer=tokenizer,
  )

# 4. Prediction
## Apply PubBiasDetect for inference
predict_test=trainer.predict(tokenized_data["inference"])

## Get the predicted class
import numpy as np
predict_test_classes=np.argmax(predict_test.predictions, axis=1)

# 5. Interpretation
## Print the predicted class
print(predict_test_classes)

## 1: Positive Results Only --> All results in the abstract are positive
## 0: Mixed and Negative Results --> At least one negative result in the abstract